In [ ]:
import os
import numpy as np
from time import sleep
from astropy.io import fits
from astropy.time import Time

In [ ]:
def get_catalog(ra, dec, target, outpath, table='swiftmastr', oformat='fits'):

    # build the command
    cmd = (f'./browse_extract_wget.pl table={table} position={ra},{dec} '
           f'outfile={outpath}/{target}/{target}xrt.fits format={oformat} radius=10')

    print(cmd)
    os.system(cmd)

In [ ]:
def get_files(target, outpath):

    # at least one file was 0 bytes... 
    try:
        cat = fits.getdata(f'{outpath}/{target}/{target}xrt.fits')
    except OSError:
        return

    if cat.shape[0] < 1:
        return

    for i in range(cat.shape[0]):
        obsid = cat['obsid'][i]
        t = Time(cat['start_time'][i], format='mjd')
        y, m = t.iso.split('-')[:2]

        wget_base = (f'wget -P {outpath}/{target} -q -nH --no-check-certificate --cut-dirs=5 '
                '-r -l0 -c -nc -np -R "index*" -erobots=off --retr-symlinks '
                f'https://heasarc.gsfc.nasa.gov/FTP/swift/data/obs/{y}_{m}/')
             
        for prod in ['xrt', 'auxil', 'log']:
            wget = f'{wget_base}{obsid}/{prod}/'
            os.system(wget)

In [ ]:
def load_PSZcatalog(unconf=False):
    from astropy.table import Table                                                       
    from numpy import append as npappend                                             

    datapath = f'{os.environ["HOME"]}/Projects/planckClusters/catalogs'
    
    ps1 = Table.read(f'{datapath}/PSZ1v2.1.fits')
    ps2 = Table.read(f'{datapath}/PSZ2v1.fits')

    # convert to pandas
    df1 = ps1.to_pandas()
    df2 = ps2.to_pandas()

    if unconf:
        # only get unconfirmed sources                                                   
        df1 = df1.loc[df1['VALIDATION'] <= 3]                                         
        df2 = df2.loc[df2['VALIDATION'] == -1]  
    
    # clean up strings -- not required
    df1 = df1.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    df2 = df2.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

    # merge the catalogs together
    df_m = df1.merge(df2, how='outer', left_on='INDEX', right_on='PSZ', suffixes=('_PSZ1', '_PSZ2'))
    
    # get the columns that we want
    cols = df_m.columns[[0, 1, 4, 5, 8, 29, 33, 34, 37, 38, 40, 51]]
    df_final = df_m[cols]

    # remerge to find bits that were missing                                        
    df_final_bigger = df_final.merge(df2, how='left', left_on='INDEX_PSZ1',         
                                 right_on='PSZ')
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:                   
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    # fill in nans                                                                  
    for col in ['NAME', 'RA', 'DEC', 'SNR', 'REDSHIFT', 'INDEX']:
        df_final_bigger[col+'_PSZ2'] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col])
    for col in ['NAME', 'RA', 'DEC']:
        df_final_bigger[col] = df_final_bigger[col+'_PSZ2'].fillna(df_final_bigger[col+'_PSZ1'])

    df_final_bigger = df_final_bigger[npappend(df_final_bigger.columns[:12].values, ['NAME', 'RA', 'DEC'])]

    return df_final_bigger
  

In [ ]:
def write_info(outpath, idx, data):
    target = data.iloc[idx]['NAME'].replace(' ', '_')
    cols = data.columns.tolist()
    vals = data.iloc[idx]
    # convert the values into all strings
    vals_str = [f'{i:.8}' if isinstance(i, float) else i for i in vals.values.tolist()]
    with open(f'{outpath}/{target}/{target}.info', 'w') as f:
        f.write(f'{",".join(cols)}\n')
        f.write(f'{",".join(vals_str)}\n')        
    

In [ ]:
#%%capture

# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

for i, (ra, dec, name) in enumerate(zip(data['RA'], data['DEC'], data['NAME'])):

    #print(name)
    name = name.replace(' ', '_')

    if not os.path.isdir(f'{outpath}/{name}'):
        os.makedirs(f'{outpath}/{name}')

    #get_catalog(ra, dec, f'{name}', outpath)
    #write_info(outpath, i, data)
    get_files(f'{name}', outpath)
    
    sleep(1)